# Install/import required modules

In [ ]:
!pip install datasets==2.10.1
#load_dataset sometimes hangs on a higher version
!pip install transformers tqdm

In [2]:
from datasets import load_dataset, load_metric

import torch
import numpy as np
import random

# other imports that will need
from transformers import DistilBertTokenizer, DistilBertConfig, DistilBertModel, get_scheduler, DistilBertPreTrainedModel, AutoTokenizer
from torch.optim import Optimizer

# additional imports I added
from torch.utils.data import DataLoader
from transformers.utils import ModelOutput
from transformers.configuration_utils import PretrainedConfig
from dataclasses import dataclass

from tqdm import tqdm
from torch.nn import Linear, Dropout, Embedding
# delete some of these when done
from typing import List, Dict, Any, Optional, Union, Tuple

# we set up some seeds so that we can reproduce results
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Model

In [4]:
@dataclass
class MyDistilBertForQAOutput(ModelOutput):
    """
    Class for outputs of MyDistilBertForQA

    Parameters:
        loss: calculated using loss function from paper
        start_logits: span-start scores pre softmax
        end_logits: span-end scores pre softmax
        type_logits: answer-type scores pre softmax
    """

    loss: Optional[torch.FloatTensor] = None
    start_logits: torch.FloatTensor = None
    end_logits: torch.FloatTensor = None
    type_logits: torch.FloatTensor = None

In [6]:
class MyDistilBertForQA(DistilBertPreTrainedModel):
    """
    MyDistilBertForQA class: slightly modified version of the DistilBertForQuestionAnswering model that also takes in and predicts the type of answer
    """
    def __init__(self, config):
        super().__init__(config)

        self.distilbert = DistilBertModel(config)

        # lin layer that will be apply transformation to hidden states in order to obtain logits for A task by transforming input tensor to have 3 output
        # dims along the last axis (dims correspond to start pos, end pos, and type)
        self.qa_lin_layer = Linear(config.dim, config.num_labels)

        self.dropout = Dropout(config.qa_dropout)

        # init weights and apply final processing
        self.post_init()

    def forward(self, input_ids, attention_mask, start_pos, end_pos, types):
        """
        Parameters (beyond the usual input_ids and attention_mask):
        start_pos: labels for index of start of answer span
        end_pos: labels for index of end of answer span
        types: labels for the type of example (i.e. short answer or no answer)
        """

        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        hidden_states = distilbert_output[0]  # (batch_size, max_query_len, dim)

        # apply lin transform to hidden states to get 3 desired output logits
        hidden_states = self.dropout(hidden_states)  # (batch_size, max_query_len, dim)
        logits = self.qa_lin_layer(hidden_states)  # (batch_size, max_query_len, 3) bc have 3 things to get

        # split into individual tensors
        start_logits, end_logits, type_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()  # (batch_size, max_query_len)
        end_logits = end_logits.squeeze(-1).contiguous()  # (batch_size, max_query_len)
        type_logits = type_logits.squeeze(-1).contiguous()

        # If we are on multi-GPU, split adds a dimension
        if len(start_pos.size()) > 1:
            start_pos = start_pos.squeeze(-1)
        if len(end_pos.size()) > 1:
            end_pos = end_pos.squeeze(-1)

        # sometimes the start/end positions are outside  model inputs -- ignore these
        ignored_index = start_logits.size(1)
        start_pos = start_pos.clamp(0, ignored_index)
        end_pos = end_pos.clamp(0, ignored_index)

        # calc loss
        log_p_start = loss_helper(start_logits, start_pos)

        log_p_end = loss_helper(end_logits, end_pos)

        log_p_type = loss_helper(type_logits, types)

        first_sum = torch.add(log_p_start, log_p_end)

        total_loss = torch.add(first_sum, log_p_type)

        return MyDistilBertForQAOutput(
            loss = total_loss,
            start_logits = start_logits,
            end_logits = end_logits,
            type_logits = type_logits)

In [7]:
def loss_helper(pred_logits, ground_truth_vals):
  """
  takes in predicted logits and ground truth values (for either start_tokens, end_tokens, or type) and returns the neg log of the softmax of the predictions
  Parameters:
  pred_logits: logits from model
  ground_truth_vals: actual values
  """

  softmax_preds = torch.softmax(pred_logits, dim=-1)

  # Use the gather function to select the softmax probability for the ground truth start index
  softmax_ground_truth = torch.gather(softmax_preds, 1, ground_truth_vals.view(-1, 1))

  neg_log = -torch.log(softmax_ground_truth)

  return neg_log.squeeze()

In [8]:
def load_model():
  """
  function to initialize tokenizer and model
  """
  # load in tokenizer and model - using cased distilled-squad version bc paper mentioned they used a model trained on squad and bc inputs are cased
  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")

  # load DistilBertConfig with num_labels = 3 because predicting 3 things (start pos, end pos, type)
  config = DistilBertConfig.from_pretrained("distilbert-base-cased-distilled-squad", num_labels=3)
  model = MyDistilBertForQA(config)

  return model, tokenizer

# Preprocessing

In [9]:
def load_data():
  """
  function to load data
  """
  data_files = {"train": "/content/drive/My Drive/CompLinguistics/all_train.json", "dev": "/content/drive/My Drive/CompLinguistics/all_dev.json"}
  dataset = load_dataset('json', data_files=data_files, chunksize=10<<23)
  return dataset['train'], dataset['dev']


In [10]:
def tokenize_entry(examples, tokenizer, is_train=True):
  """
  tokenizes a batch of entries and converts span starts/ends accordingly

  Parameters:
  examples: batch of examples from dataset
  tokenizer: HuggingFace tokenizer
  is_train : boolean denoting whether or not the examples being passed in come from the train or validation dataset

  Returns:
  tokenized entries with type and converted span start/end
  """
  # get all the questions into a list -- in the dictionary, have list of questions where each "question" is a list with 1 dictionary in it
  questions = [examples['questions'][i][0]['input_text'] for i in range(len(examples['questions']))]
  contexts = [examples['contexts'][i] for i in range(len(examples['contexts']))]

  # only options for type are 4 and 0 bc data only has short and no answer
  types = [4 if examples['answers'][i][0]['input_text'] == 'no-answer' else 0 for i in range(len(examples['answers']))]
  answers = examples['answers']

  # tokenize question and context, return offsets so can map from character to token indices for span
  # set max length to 512 and truncation to true to account for contexts that are too long
  tokenized_egs = tokenizer(
        questions,
        contexts,
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

  tokenized_egs["start_pos"] = []
  tokenized_egs["end_pos"] = []

  # turn types into tensors so can be used in model
  tokenized_egs["types"] = [torch.tensor(eg_type, dtype=torch.long) for eg_type in types]

  # get offset mappings so can map from token to char indices and compute start and end of span
  offset_mapping = tokenized_egs.pop("offset_mapping")

  start_pos = []
  end_pos = []

  # create offset_mappings list so can add to tokenized_egs if tokenizing validation dataset)
  offset_mappings = []

  for i, offset in enumerate(offset_mapping):
        # in the dictionary, have list of answers where each "answer" is a list with 1 dictionary in it
        ans = answers[i][0]

        start_char = ans["span_start"]
        end_char = ans["span_end"]

        # get sequence corresponding to current example (so can know what is context vs what is question)
        seq_ids = tokenized_egs.sequence_ids(i)

        # Find start and end of context
        idx = 0
        while seq_ids[idx] != 1:
            idx += 1
        context_start = idx

        while seq_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If ans not fully inside context, label it (0, 0) and CHANGE TYPE TO NO_ANSWER
        # if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_pos.append(0)
            end_pos.append(0)
            types[i] = 4

        # if marked as "no answer" then also append 0 (for CLS token)
        elif ans['input_text'] == 'no-answer':
            tokenized_egs["start_pos"].append(0)
            tokenized_egs["end_pos"].append(0)

        else:
            # Otherwise idx = start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_pos.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_pos.append(idx + 1)

        # if working with validation dataset, then also set offsets corresponding to question to -1 (to ensure wont be in actual context)
        # so will be able to map start/end tokens back to correct span of context during eval
        if not is_train:
            new_offset = [o if seq_ids[k] == 1 else [-1, -1] for k, o in enumerate(offset)]
            offset_mappings.append(new_offset)


  # add lists of start positions, end positions, and types to output dict
  tokenized_egs["start_pos"] = start_pos
  tokenized_egs["end_pos"] = end_pos
  tokenized_egs['types'] = types

  # if working with validation dataset, add offset mappings to tokenized_egs
  if not is_train:
    tokenized_egs['offset_mappings'] = offset_mappings


  return tokenized_egs

In [11]:
def preprocess_and_tokenize(dataset, tokenizer, batch_size, is_train=True):
  """
  tokenizes entire dataset by passing batches into tokenize_entry function

  Parameters:
  dataset
  tokenizer
  is_train : boolean denoting whether or not the examples being passed in come from the train or validation dataset

  Returns:
  dataloader of tokenized entries
  """

  tokenized_dataset = dataset.map(lambda examples: tokenize_entry(examples, tokenizer, is_train), batched = True)

  # shuffle because datasets are ordered with short answer first and no answer second
  return DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)


# Training and Evaluating

In [12]:
def eval_for_train_loop(model, device, validation_dataloader):
  """
  evaluates model using validation dataset and returns loss from last layer

  Parameters:
  model : trained model
  device : device that model is on
  validation_dataloader : dataLoader with validation examples
  """

  # put model in eval
  model.eval()

  # list to store predicted answers
  pred_answers = []
  actual_answers = []

  for batch in validation_dataloader:

    # get all of the components that need to pass into model.
    inputs = torch.stack(batch['input_ids'], dim=1).to(device)
    start_pos = batch['start_pos'].to(device)
    end_pos = batch['end_pos'].to(device)
    types = batch['types'].to(device)
    mask = torch.stack(batch['attention_mask'], dim=1).to(device)

    with torch.no_grad():
      out = model(input_ids = inputs, attention_mask = mask, start_pos = start_pos, end_pos = end_pos, types=types)

    loss = torch.mean(out.loss)

  # return loss from final layer
  return loss


In [13]:
def train_loop(model, optimizer, num_epochs, train_dataloader, validation_dataloader, lr_scheduler, device):
  """
  trains a model by performing forward pass and backpropagating on batches to optimize loss (using loss function from paper)

  Parameters:
  model : model to be trained
  optimizer : training optimizer
  num_epochs : number of epochs to train for
  train_dataloader : dataLoader with training examples
  validation_dataloader : dataLoader with validation examples
  lr_scheduler : learning rate scheduler
  device : device that training will be performed on
  """

  train_losses = []
  val_losses = []

  for epoch in range(num_epochs):
    # put the model in training mode (important that this is done each epoch,
    # since we put the model into eval mode during validation)
    model.train()

    print(f"Epoch {epoch + 1} training:")
    progress_bar = tqdm(range(len(train_dataloader)))

    for batch in train_dataloader:

      # zero out the gradients so that gradient isn't combo of already-used old gradient and newly-computed gradient
      optimizer.zero_grad()

      # get all of the components that need to pass into model.
      inputs = torch.stack(batch['input_ids'], dim=1).to(device)
      start_pos = batch['start_pos'].to(device)
      end_pos = batch['end_pos'].to(device)
      types = batch['types'].to(device)
      mask = torch.stack(batch['attention_mask'], dim=1).to(device)

      # pass in labels in addition to tweets and attn_mask so can get loss
      out = model(input_ids = inputs, attention_mask = mask, start_pos = start_pos, end_pos = end_pos, types=types)

      # get predicts for start and end tokens and types
      start_preds = torch.argmax(out.start_logits, dim=-1)
      end_preds = torch.argmax(out.end_logits, dim=-1)
      types_preds = torch.argmax(out.type_logits, dim=-1)

      # get loss (and take avg bc have different losses for each thing in batch) and backprop
      loss = torch.mean(out.loss)
      loss.backward()

      # update optimizer and learning rate
      optimizer.step()
      lr_scheduler.step()

      # # update progress bar
      progress_bar.update(1)

    # get losses from last layers for train and validation and add to lists
    train_losses.append(loss.item())
    val_loss = eval_for_train_loop(model, device, validation_dataloader)
    val_losses.append(val_loss.item())

  return train_losses, val_losses

In [14]:
def get_best_answers(start_logits, end_logits, validation_batch, n_best_size= 20):
  """
  takes in start and end logits, finds "best answer" according to score function from paper, and uses offset mappings to map said "best" start and end tokens
  back a span of text from context

  Parameters:
  start_logits: span-start scores from model
  end_logits: span-end scores from model
  validation_batch: current batch of validation data from validation_data_loader being evaluated
  n_best_size: int limiting how number of start/end logits considered in argmax (default = 20)

  Returns:
  text corresponding to "best answer" for each example in current batch of validation data
  """

  pred_answers = []
  for idx in range(len(validation_batch['contexts'])):

      # get context for current example
      context = validation_batch['contexts'][idx]

      answers = []

      # get start logit, end logit, and offsets associated with specific feature
      start_logit = start_logits[idx]
      end_logit = end_logits[idx]

      start_is_CLS = start_logits[idx, 0]  # get logit associated with the start token being the CLS token (i.e. the token in the 0th position)
      end_is_CLS = end_logits[idx, 0]  # get logit associated with the end token being the CLS token (i.e. the token in the 0th position)

      offsets = validation_batch["offset_mappings"]

      start_indices = np.argsort(start_logit)[-1 : -n_best_size - 1 : -1].tolist()
      end_indices = np.argsort(end_logit)[-1 : -n_best_size - 1 : -1].tolist()

      for start_idx in start_indices:
          for end_idx in end_indices:
              # offsets[start_idx] = word corresponding to start index, [0]= get starting token for that word, [idx] = get starting token for this logit
              if offsets[start_idx][0][idx] == -1 or offsets[end_idx][1][idx] == -1:
                  continue
              # skip invalid answers where end token is before start token
              if end_idx < start_idx:
                  continue

              # compute score as defined in paper: f_start(s, c) + f_end(e, c) = f_start(s = CLS, c) - f_end(e = CLS, c)
              score = start_logit[start_idx] + end_logit[end_idx] - start_is_CLS - end_is_CLS
              ans = {
                  "text": context[offsets[start_idx][0][idx] : offsets[end_idx][1][idx]],
                  "score": score
              }
              answers.append(ans)


      # select the answer with best score
      if len(answers) > 0:
          best_ans = max(answers, key=lambda x: x["score"])
          # print(best_ans['text'])
          pred_answers.append(best_ans["text"])
      # if have no valid answers (which will need if type is "no-answer"), then "best answer" is empty string
      else:
          pred_answers.append("")

  return pred_answers


In [16]:
def calc_metrics(pred_spans, actual_spans):
  """
  uses predicted and ground truth answer spans to calculate precision, recall, and F1 score as accuracy metrics

  Parameters:
  pred_spans: predicted spans of text using "best" start/end logits from model
  actual_spans: actual spans of text

  Returns:
  precision, recall, and F1 score
  """
  precisions = []
  recalls = []
  f1_scores = []

  for pred_span, act_span in zip(pred_spans, actual_spans):
    # if both predicted and actual spans are no answer, then precision, recall, and F1 are all perfect (i.e. 1)
    if len(pred_span) == 0 and len(act_span) == 0:
      precisions.append(1)
      recalls.append(1)
      f1_scores.append(1)

    # if predicted span is no answer but actual span is short answer, then precision, recall, and F1 are all 0 (because have no true positives)
    elif len(pred_span) == 0 and len(act_span) > 0:
      precisions.append(0)
      recalls.append(0)
      f1_scores.append(0)

    # similarly if predicted span is short answer but actual span is no answer, then precision, recall, and F1 are all 0 (because have no true positives)
    elif len(pred_span) > 0 and len(act_span) == 0:
      precisions.append(0)
      recalls.append(0)
      f1_scores.append(0)

    # if both predicted and actual spans are short answer
    else:
      true_pos = sum(1 for token in pred_span if token in act_span) # true pos = number of tokens in both predicted and actual span
      false_pos = sum(1 for token in pred_span if token not in act_span) # false pos = number of tokens in predicted but not actual span
      false_negs = sum(1 for token in act_span if token not in pred_span) # false negs = number of tokens in actual but not predicted span

      p = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
      precisions.append(p)
      r = true_pos / (true_pos + false_negs) if (true_pos + false_negs) > 0 else 0
      recalls.append(r)
      f1 = 2 * (p*r/(p + r)) if (p + r) > 0 else 0
      f1_scores.append(f1)

  precision = np.mean(precisions)
  recall = np.mean(recalls)
  f1 = np.mean(f1_scores)

  return precision, recall, f1

In [17]:
def eval_loop(model, validation_data_loader, device, n_best_size= 20):
  """
  evaluates model on validation dataset, using precision, recall, and F1 score as accuracy metrics

  Parameters:
  model: model to evaluate
  device: device to put the model and tensors on
  validation_data_loader: validation data in dataloader
  n_best_size: int limiting how number of start/end logits considered in argmax (default = 20)

  Returns:
  precision, recall, and F1 score
  """

  # put model in eval
  model.eval()

  # we like progress bars :)
  progress_bar = tqdm(range(len(validation_data_loader)))

  # list to store predicted answers
  pred_answers = []
  actual_answers = []

  for batch in validation_data_loader:

    # get all of the components that need to pass into model.
    inputs = torch.stack(batch['input_ids'], dim=1).to(device)
    start_pos = batch['start_pos'].to(device)
    end_pos = batch['end_pos'].to(device)
    types = batch['types'].to(device)
    mask = torch.stack(batch['attention_mask'], dim=1).to(device)

    with torch.no_grad():
      out = model(input_ids = inputs, attention_mask = mask, start_pos = start_pos, end_pos = end_pos, types=types)

    start_logits = out.start_logits.cpu().numpy()
    end_logits = out.end_logits.cpu().numpy()
    type_logits = out.type_logits.cpu().numpy()

    best_pred_answers = get_best_answers(start_logits, end_logits, batch)

    # get span text from actual answers
    batch_actual_answers = batch['answers'][0]['span_text']

    # add to fulls lists of predicted/actual answers
    pred_answers = pred_answers + best_pred_answers
    actual_answers = actual_answers + batch_actual_answers

    progress_bar.update(1)

  precision, recall, f1 = calc_metrics(pred_answers, actual_answers)
  return precision, recall, f1

In [18]:
def main():
  '''Here's the basic structure of the main block -- feel free to add or
  remove parameters/helper functions as you see fit, but all steps here are
  needed and we expect to see precision, recall, and f1 scores printed out'''
  device = "cuda" if torch.cuda.is_available() else "cpu"
  batch_size = 16 # smaller batch size to account for issues with GPU credit availablity

  model, tokenizer = load_model()
  model.to(device)


  train, validation = load_data()

  train_data_loader = preprocess_and_tokenize(train, tokenizer, batch_size)
  validation_data_loader = preprocess_and_tokenize(validation, tokenizer, batch_size, is_train = False)

  num_epochs = 1 # as per the paper

  # instantiate AdamW optimizer on model parameters with learning rate of 3e-5 (as per the paper)
  optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

  # set up the learning rate scheduler
  lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=50,
    num_training_steps=len(train_data_loader) * num_epochs
  )

  train_losses, val_losses = train_loop(model, optimizer, num_epochs,
          train_data_loader, validation_data_loader,
          lr_scheduler, device)

  precision, recall, f1_score = eval_loop(model, validation_data_loader, device)

  # added print lines for train and validation losses
  print("TRAIN LOSSES (from train loop)", train_losses)
  print("VALIDATION LOSSES (from train loop)", val_losses)
  print("PRECISION: ", precision)
  print("RECALL: ", recall)
  print("F1-SCORE: ", f1_score)

if __name__ == "__main__":
  main()

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Map:   0%|          | 0/27866 [00:00<?, ? examples/s]

Map:   0%|          | 0/1743 [00:00<?, ? examples/s]

Epoch 1 training:


100%|██████████| 109/109 [00:48<00:00,  2.25it/s]

TRAIN LOSSES (from train loop) [7.328113079071045]
VALIDATION LOSSES (from train loop) [4.823604583740234]
PRECISION:  0.2913628098593942
RECALL:  0.37559986233440834
F1-SCORE:  0.2902818236083016
